In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-09 04:12:02.922911: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=None)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_1_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

Epoch 24/1000
10/10 [==============================] - 0s 20ms/step - loss: 10.6153 - mean_squared_error: 10.6153 - rmse: 3.2581 - mean_absolute_error: 2.6312 - val_loss: 11.0868 - val_mean_squared_error: 11.0868 - val_rmse: 3.3297 - val_mean_absolute_error: 2.4472 - lr: 0.0010
Epoch 25/1000
10/10 [==============================] - 0s 21ms/step - loss: 10.9885 - mean_squared_error: 10.9885 - rmse: 3.3149 - mean_absolute_error: 2.6605 - val_loss: 10.6381 - val_mean_squared_error: 10.6381 - val_rmse: 3.2616 - val_mean_absolute_error: 2.4035 - lr: 0.0010
Epoch 26/1000
10/10 [==============================] - 0s 23ms/step - loss: 10.6230 - mean_squared_error: 10.6230 - rmse: 3.2593 - mean_absolute_error: 2.6166 - val_loss: 9.5125 - val_mean_squared_error: 9.5125 - val_rmse: 3.0842 - val_mean_absolute_error: 2.2804 - lr: 0.0010
Epoch 27/1000
10/10 [==============================] - 0s 22ms/step - loss: 10.4618 - mean_squared_error: 10.4618 - rmse: 3.2345 - mean_absolute_error: 2.6323 - val_

Epoch 54/1000
10/10 [==============================] - 0s 19ms/step - loss: 8.9509 - mean_squared_error: 8.9509 - rmse: 2.9918 - mean_absolute_error: 2.3971 - val_loss: 7.7767 - val_mean_squared_error: 7.7767 - val_rmse: 2.7887 - val_mean_absolute_error: 2.0708 - lr: 0.0010
Epoch 55/1000
10/10 [==============================] - 0s 19ms/step - loss: 10.6832 - mean_squared_error: 10.6832 - rmse: 3.2685 - mean_absolute_error: 2.6258 - val_loss: 13.3717 - val_mean_squared_error: 13.3717 - val_rmse: 3.6567 - val_mean_absolute_error: 2.7386 - lr: 0.0010
Epoch 56/1000
10/10 [==============================] - 0s 19ms/step - loss: 9.0791 - mean_squared_error: 9.0791 - rmse: 3.0132 - mean_absolute_error: 2.3902 - val_loss: 11.2038 - val_mean_squared_error: 11.2038 - val_rmse: 3.3472 - val_mean_absolute_error: 2.4481 - lr: 0.0010
Epoch 57/1000
10/10 [==============================] - 0s 19ms/step - loss: 8.0516 - mean_squared_error: 8.0516 - rmse: 2.8375 - mean_absolute_error: 2.2693 - val_loss: 

Epoch 84/1000
10/10 [==============================] - 0s 20ms/step - loss: 8.7297 - mean_squared_error: 8.7297 - rmse: 2.9546 - mean_absolute_error: 2.3610 - val_loss: 7.1589 - val_mean_squared_error: 7.1589 - val_rmse: 2.6756 - val_mean_absolute_error: 1.9719 - lr: 0.0010
Epoch 85/1000
10/10 [==============================] - 0s 20ms/step - loss: 7.9590 - mean_squared_error: 7.9590 - rmse: 2.8212 - mean_absolute_error: 2.2472 - val_loss: 8.5097 - val_mean_squared_error: 8.5097 - val_rmse: 2.9171 - val_mean_absolute_error: 2.1230 - lr: 0.0010
Epoch 86/1000
10/10 [==============================] - 0s 20ms/step - loss: 8.9799 - mean_squared_error: 8.9799 - rmse: 2.9966 - mean_absolute_error: 2.3920 - val_loss: 13.1229 - val_mean_squared_error: 13.1229 - val_rmse: 3.6226 - val_mean_absolute_error: 2.7602 - lr: 0.0010
Epoch 87/1000
10/10 [==============================] - 0s 24ms/step - loss: 7.0596 - mean_squared_error: 7.0596 - rmse: 2.6570 - mean_absolute_error: 2.1032 - val_loss: 7.50

Epoch 113/1000
10/10 [==============================] - 0s 21ms/step - loss: 6.4640 - mean_squared_error: 6.4640 - rmse: 2.5424 - mean_absolute_error: 1.9956 - val_loss: 7.8356 - val_mean_squared_error: 7.8356 - val_rmse: 2.7992 - val_mean_absolute_error: 2.0741 - lr: 5.0000e-04
Epoch 114/1000
10/10 [==============================] - 0s 21ms/step - loss: 6.6080 - mean_squared_error: 6.6080 - rmse: 2.5706 - mean_absolute_error: 2.0238 - val_loss: 10.6620 - val_mean_squared_error: 10.6620 - val_rmse: 3.2653 - val_mean_absolute_error: 2.4775 - lr: 5.0000e-04
Epoch 115/1000
10/10 [==============================] - 0s 20ms/step - loss: 5.8219 - mean_squared_error: 5.8219 - rmse: 2.4129 - mean_absolute_error: 1.9043 - val_loss: 9.6433 - val_mean_squared_error: 9.6433 - val_rmse: 3.1054 - val_mean_absolute_error: 2.3370 - lr: 5.0000e-04
Epoch 116/1000
10/10 [==============================] - 0s 21ms/step - loss: 6.2711 - mean_squared_error: 6.2711 - rmse: 2.5042 - mean_absolute_error: 1.9779 

...

Epoch 171/1000
10/10 [==============================] - 0s 20ms/step - loss: 5.8724 - mean_squared_error: 5.8724 - rmse: 2.4233 - mean_absolute_error: 1.8771 - val_loss: 9.0383 - val_mean_squared_error: 9.0383 - val_rmse: 3.0064 - val_mean_absolute_error: 2.2883 - lr: 7.8125e-06
Epoch 172/1000
10/10 [==============================] - 0s 21ms/step - loss: 6.1295 - mean_squared_error: 6.1295 - rmse: 2.4758 - mean_absolute_error: 1.9459 - val_loss: 9.2961 - val_mean_squared_error: 9.2961 - val_rmse: 3.0490 - val_mean_absolute_error: 2.3256 - lr: 7.8125e-06
Epoch 173/1000
10/10 [==============================] - 0s 20ms/step - loss: 5.8070 - mean_squared_error: 5.8070 - rmse: 2.4098 - mean_absolute_error: 1.8760 - val_loss: 9.4197 - val_mean_squared_error: 9.4197 - val_rmse: 3.0692 - val_mean_absolute_error: 2.3428 - lr: 7.8125e-06
Epoch 174/1000
10/10 [==============================] - 0s 20ms/step - loss: 5.8596 - mean_squared_error: 5.8596 - rmse: 2.4207 - mean_absolute_error: 1.8719 - 

10/10 [==============================] - 0s 21ms/step - loss: 14.0427 - mean_squared_error: 14.0427 - rmse: 3.7474 - mean_absolute_error: 2.9965 - val_loss: 13.8870 - val_mean_squared_error: 13.8870 - val_rmse: 3.7265 - val_mean_absolute_error: 2.6940 - lr: 0.0010
Epoch 10/1000
10/10 [==============================] - 0s 21ms/step - loss: 13.1415 - mean_squared_error: 13.1415 - rmse: 3.6251 - mean_absolute_error: 2.9195 - val_loss: 11.8872 - val_mean_squared_error: 11.8872 - val_rmse: 3.4478 - val_mean_absolute_error: 2.5339 - lr: 0.0010
Epoch 11/1000
10/10 [==============================] - 0s 23ms/step - loss: 12.6497 - mean_squared_error: 12.6497 - rmse: 3.5566 - mean_absolute_error: 2.8378 - val_loss: 10.5317 - val_mean_squared_error: 10.5317 - val_rmse: 3.2453 - val_mean_absolute_error: 2.5014 - lr: 0.0010
Epoch 12/1000
10/10 [==============================] - 0s 19ms/step - loss: 14.6376 - mean_squared_error: 14.6376 - rmse: 3.8259 - mean_absolute_error: 3.0242 - val_loss: 10.694

Epoch 38/1000
10/10 [==============================] - 0s 20ms/step - loss: 11.4121 - mean_squared_error: 11.4121 - rmse: 3.3782 - mean_absolute_error: 2.7141 - val_loss: 20.4464 - val_mean_squared_error: 20.4464 - val_rmse: 4.5218 - val_mean_absolute_error: 3.4252 - lr: 5.0000e-04
Epoch 39/1000
10/10 [==============================] - 0s 20ms/step - loss: 10.9828 - mean_squared_error: 10.9828 - rmse: 3.3140 - mean_absolute_error: 2.6740 - val_loss: 18.8851 - val_mean_squared_error: 18.8851 - val_rmse: 4.3457 - val_mean_absolute_error: 3.2567 - lr: 5.0000e-04
Epoch 40/1000
10/10 [==============================] - 0s 20ms/step - loss: 11.2324 - mean_squared_error: 11.2324 - rmse: 3.3515 - mean_absolute_error: 2.7021 - val_loss: 17.7759 - val_mean_squared_error: 17.7759 - val_rmse: 4.2162 - val_mean_absolute_error: 3.1378 - lr: 5.0000e-04
Epoch 41/1000
10/10 [==============================] - 0s 21ms/step - loss: 11.1581 - mean_squared_error: 11.1581 - rmse: 3.3404 - mean_absolute_error:

10/10 [==============================] - 0s 21ms/step - loss: 9.3935 - mean_squared_error: 9.3935 - rmse: 3.0649 - mean_absolute_error: 2.4741 - val_loss: 14.6087 - val_mean_squared_error: 14.6087 - val_rmse: 3.8221 - val_mean_absolute_error: 2.8472 - lr: 2.5000e-04
Epoch 67/1000
10/10 [==============================] - 0s 20ms/step - loss: 9.2988 - mean_squared_error: 9.2988 - rmse: 3.0494 - mean_absolute_error: 2.4350 - val_loss: 12.8110 - val_mean_squared_error: 12.8110 - val_rmse: 3.5792 - val_mean_absolute_error: 2.6489 - lr: 2.5000e-04
Epoch 68/1000
10/10 [==============================] - 0s 20ms/step - loss: 9.3159 - mean_squared_error: 9.3159 - rmse: 3.0522 - mean_absolute_error: 2.4389 - val_loss: 11.6928 - val_mean_squared_error: 11.6928 - val_rmse: 3.4195 - val_mean_absolute_error: 2.5329 - lr: 2.5000e-04
Epoch 69/1000
10/10 [==============================] - 0s 20ms/step - loss: 9.9965 - mean_squared_error: 9.9965 - rmse: 3.1617 - mean_absolute_error: 2.5051 - val_loss: 11

Epoch 94/1000
10/10 [==============================] - 0s 19ms/step - loss: 8.1971 - mean_squared_error: 8.1971 - rmse: 2.8631 - mean_absolute_error: 2.2785 - val_loss: 11.6156 - val_mean_squared_error: 11.6156 - val_rmse: 3.4082 - val_mean_absolute_error: 2.5416 - lr: 6.2500e-05
Epoch 95/1000
10/10 [==============================] - 0s 20ms/step - loss: 7.8917 - mean_squared_error: 7.8917 - rmse: 2.8092 - mean_absolute_error: 2.2264 - val_loss: 11.5777 - val_mean_squared_error: 11.5777 - val_rmse: 3.4026 - val_mean_absolute_error: 2.5389 - lr: 6.2500e-05
Epoch 96/1000
10/10 [==============================] - 0s 19ms/step - loss: 8.1400 - mean_squared_error: 8.1400 - rmse: 2.8531 - mean_absolute_error: 2.2786 - val_loss: 12.3481 - val_mean_squared_error: 12.3481 - val_rmse: 3.5140 - val_mean_absolute_error: 2.6397 - lr: 6.2500e-05
Epoch 97/1000
10/10 [==============================] - 0s 20ms/step - loss: 7.8998 - mean_squared_error: 7.8998 - rmse: 2.8107 - mean_absolute_error: 2.2167 

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)